In [1]:
import os
from tqdm import tqdm
import pandas as pd
import numpy as np
import torch
from code_function import preprocess_script, make_dataset
from glob import glob
import subprocess
import re
import shutil

In [4]:
def Pav(lines):
    vline = []
    loop_on = 0
    for_num = 0
    variable_list = ''
    for line in lines:
        start = line.lstrip()
        Blank = len(line) - len(start)
        Blank = line[:Blank]
        if (start.startswith("for")) or (start.startswith("while")):
            if not loop_on:
                loop_on = 1
                for_num += 1
                line = line + f"#{for_num}"
            else:
                if variable_list:
                    vline.append(f"{Blank}print('{for_num} {variable_list}:',{variable_list[:-3]})\n{Blank}")
                    for_num += 1
                    line = line + f"#{for_num}"
                else:
                    pass
        else:
            if loop_on:
                if variable_list:
                    vline.append(f"{Blank}print('{for_num} {variable_list}:',{variable_list[:-3]})\n{Blank}")
                    loop_on = 0
                else:
                    pass
            else:
                pass
        
        
        if "=" in line:
            index = line.find("=")
            vb = line[:index]
            f = vb.strip()
            if "," in f:
                FC = f.split(",")
                FCB = len(FC)
                tmp_num = 0
                for fc in FC:
                    fc = fc.strip()
                    c = re.findall(r'\w', fc)
                    if len(fc) == len(c):
                        tmp_num += 1
                if FCB == tmp_num:
                    variable = vb.lstrip()
                    blank = len(vb) - len(variable)
                    blank = vb[:blank]
                    vline.append(line)
                    rep = f"{blank}print('{variable}:',{variable})"
                    vline.append(rep)
                else:
                    vline.append(line)
            else:        
                c = re.findall(r'\w', f)
                d = re.findall(r'\w+\[\w+\]', f)
                if len(f) == len(c):
                    variable = vb.lstrip()
                    blank = len(vb) - len(variable)
                    blank = vb[:blank]
                    vline.append(line)
                    rep = f"{blank}print('{variable}:',{variable})"
                    variable_list += f"{variable}, "
                    vline.append(rep)
                elif len(f) == len(d):
                    variable = vb.lstrip()
                    blank = len(vb) - len(variable)
                    blank = vb[:blank]
                    vline.append(line)
                    rep = f"{blank}print('{variable}:',{variable})"
                    variable_list += f"{variable}, "
                    vline.append(rep)
                else:
                    vline.append(line)
        else:
            vline.append(line)
    vline = "\n".join(vline)
    return vline

In [5]:
#python3코드
problems= sorted(glob('D:/open/code/*/*.py'))
for problem in tqdm(problems):
    name = os.path.basename(problem)
    name = name.split(".")[0]
    folder = name.split("_")[0]
    lines = preprocess_script(problem)
#     lines = text.split("\n")
    text = Pav(lines)
    if not os.path.exists(f"D:/code_preprocessing/original/{folder}"):
        os.makedirs(f"D:/code_preprocessing/original/{folder}")
    with open(f'D:/code_preprocessing/original/{folder}/{name}.py', "wt", encoding='utf-8') as file:
        try:
            file.write(text)
        except:
            print(text)
            break

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 45101/45101 [04:02<00:00, 185.99it/s]


In [6]:
b1 = [['1','1','1','1','1','1','1','1','1','1'],['1 1', '1','1','1'],['1', '1 1','1 1','1 1'],['1 1','1 1','1','1'],['1','1','1 1','1 1'],['1 1 1'],['1 1 1','1'],['1','1 1 1'],['1 1 1 1'],['1 1 1 1','1'],['1','1 1 1 1'],['1 1 1 1 1'],['1 1 1 1 1 1']]
def inputs(input_data):
    x = f"for FFF in {input_data}:"
    return x
x = inputs(b1)

In [7]:
#생성 후 실행하는 코드
# problems= sorted(glob('D:/code_preprocessing/original/*/*.py'))
#이후
problems= sorted(glob('D:/code_preprocessing/non_executable/*.py'))
for problem in tqdm(problems):
    name = os.path.basename(problem)
    name = name.split(".")[0]
    folder = name.split("_")[0]
    re_lines=[]
    with open(problem,'r',encoding='utf-8') as file:
        lines = file.readlines()
        for line in lines:
            re_lines.append("            "+line)
    text = '\n'.join(re_lines)
    text =  f"import sys\nsys.stdout = open(f'D:/code_preprocessing/results/{name}.txt', 'w')\nfrom unittest.mock import patch\n"+x+"\n    try:\n        with patch('builtins.input') as input_mock:\n            input_mock.side_effect = FFF\n            print('**START**')\n"+text+"\n        print('GOODJOBANDSUCCESS')\n    except Exception as e:\n        print('####')\nsys.stdout.close()"
        #stdin별도처리
    if "stdin" in text:
        with open(f"D:/code_preprocessing/execute/{name}.py", "wt", encoding='utf-8') as f:
            f.write(text)
    elif ("stdin" in text) and ("input()" not in text):
        with open(f"D:/code_preprocessing/execute/{name}.py", "wt", encoding='utf-8') as f:
            text = f"import sys\nsys.stdin=open('D:/open/temp/temp.txt')\nfrom sys import stdin\nstdin=open('D:/open/temp/temp.txt')\nsys.stdout = open('D:/code_preprocessing/results/{name}.txt', 'w')\n" + text +"\nsys.stdout.close()"
            f.write(text)
        try:
            p = subprocess.run(['python', f"D:/code_preprocessing/execute/{name}.py"], timeout=1.5)
        except:
            continue       
    else:
        with open(f"D:/code_preprocessing/execute/{name}.py", "wt", encoding='utf-8') as f:
            f.write(text)
        try:
            p = subprocess.run(['python', f"D:/code_preprocessing/execute/{name}.py"], timeout=1.5)
        except:
            continue

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 45101/45101 [1:49:11<00:00,  6.88it/s]


In [11]:
problems= sorted(glob('D:/code_preprocessing/original/*/*.py'))
for problem in tqdm(problems):
    name = os.path.basename(problem)
    name = name.split(".")[0]
    folder = name.split("_")[0]
    if not os.path.exists(f"D:/code_preprocessing/results/{name}.txt"):
        shutil.copyfile(f"D:/code_preprocessing/execute/{name}.py", f"D:/code_preprocessing/non_executable/{name}.py")
    else:
        with open(f"D:/code_preprocessing/results/{name}.txt", "r") as f:
            text = f.read()
        if 'GOODJOBANDSUCCESS' in text:
            shutil.copyfile(f"D:/code_preprocessing/execute/{name}.py", f"D:/code_preprocessing/executable/{name}.py")
            with open(f"D:/code_preprocessing/results/{name}.txt", "r") as f:
                text = f.read()
            lines= text.split("\n")
            text_data = []
            for line in lines:
                if line == "**START**":
                    pass
                elif line == "####":
                    text_data = []
                elif line == "GOODJOBANDSUCCESS":
                    text_data = "\n".join(text_data)
                    with open(f"D:/code_preprocessing/executable/{name}.txt", "wt") as f:
                        f.write(text_data)
                    break
                else:
                    if line in text_data:
                        pass
                    else:
                        text_data.append(line)

        else:
            shutil.copyfile(f"D:/code_preprocessing/execute/{name}.py", f"D:/code_preprocessing/non_executable/{name}.py")

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 45101/45101 [47:49<00:00, 15.72it/s]


In [12]:
def Pav(lines):
    vline = []
    loop_on = 0
    for_num = 0
    variable_list = ''
    for line in lines:
        start = line.lstrip()
        Blank = len(line) - len(start)
        Blank = line[:Blank]
        if (start.startswith("for")) or (start.startswith("while")):
            if not loop_on:
                loop_on = 1
                for_num += 1
                line = line + f"#{for_num}"
            else:
                if variable_list:
                    vline.append(f"{Blank}try:\n{Blank}    print('{for_num} {variable_list}:',{variable_list[:-3]})\n{Blank}except:\n{Blank}    pass")
                    for_num += 1
                    line = line + f"#{for_num}"
                else:
                    pass
        else:
            if loop_on:
                if variable_list:
                    vline.append(f"{Blank}try:\n{Blank}    print('{for_num} {variable_list}:',{variable_list[:-3]})\n{Blank}except:\n{Blank}    pass")
                    loop_on = 0
                else:
                    pass
            else:
                pass
        
        
        if "=" in line:
            index = line.find("=")
            vb = line[:index]
            f = vb.strip()
            if "," in f:
                FC = f.split(",")
                FCB = len(FC)
                tmp_num = 0
                for fc in FC:
                    fc = fc.strip()
                    c = re.findall(r'\w', fc)
                    if len(fc) == len(c):
                        tmp_num += 1
                if FCB == tmp_num:
                    variable = vb.lstrip()
                    blank = len(vb) - len(variable)
                    blank = vb[:blank]
                    vline.append(line)
                    rep = f"{blank}print('{variable}:',{variable})"
                    vline.append(rep)
                else:
                    vline.append(line)
            else:        
                c = re.findall(r'\w', f)
                d = re.findall(r'\w+\[\w+\]', f)
                if len(f) == len(c):
                    variable = vb.lstrip()
                    blank = len(vb) - len(variable)
                    blank = vb[:blank]
                    vline.append(line)
                    rep = f"{blank}print('{variable}:',{variable})"
                    variable_list += f"{variable}, "
                    vline.append(rep)
                elif len(f) == len(d):
                    variable = vb.lstrip()
                    blank = len(vb) - len(variable)
                    blank = vb[:blank]
                    vline.append(line)
                    rep = f"{blank}print('{variable}:',{variable})"
                    variable_list += f"{variable}, "
                    vline.append(rep)
                else:
                    vline.append(line)
        else:
            vline.append(line)
    vline = "\n".join(vline)
    return vline

In [13]:
#python3코드
problems= sorted(glob('D:/open/code/*/*.py'))
for problem in tqdm(problems):
    name = os.path.basename(problem)
    name = name.split(".")[0]
    folder = name.split("_")[0]
    text = preprocess_script(problem)
    lines = text.split("\n")
    text = Pav(lines)
    if not os.path.exists(f"D:/code_preprocessing/original/{folder}"):
        os.makedirs(f"D:/code_preprocessing/original/{folder}")
    with open(f'D:/code_preprocessing/original/{folder}/{name}.py', "wt", encoding='utf-8') as file:
        try:
            file.write(text)
        except:
            print(text)
            break

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 45101/45101 [00:33<00:00, 1358.22it/s]


In [14]:
#executable 파일에 없는 데이터를 non_executable 데이터로 만들기
problems= sorted(glob('D:/code_preprocessing/execute/*.py'))
for problem in tqdm(problems):
    name = os.path.basename(problem)
    name = name.split(".")[0]
    folder = name.split("_")[0]
    if not os.path.exists(f"D:/code_preprocessing/executable/{name}.txt"):
        shutil.copyfile(f"D:/code_preprocessing/original/{folder}/{name}.py", f"D:/code_preprocessing/non_executable/{name}.py")
        

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 45101/45101 [01:47<00:00, 419.34it/s]


In [15]:
#생성된 txt지우기
files = glob(f"D:/code_preprocessing/results/*.txt")
for f in files:
    os.remove(f)

In [16]:
#생성 후 실행하는 코드
# problems= sorted(glob('D:/code_preprocessing/original/*/*.py'))
#이후
problems= sorted(glob('D:/code_preprocessing/non_executable/*.py'))
for problem in tqdm(problems):
    name = os.path.basename(problem)
    name = name.split(".")[0]
    folder = name.split("_")[0]
    re_lines=[]
    with open(problem,'r',encoding='utf-8') as file:
        lines = file.readlines()
        for line in lines:
            re_lines.append("            "+line)
    text = '\n'.join(re_lines)
    text =  f"import sys\nsys.stdout = open(f'D:/code_preprocessing/results/{name}.txt', 'w')\nfrom unittest.mock import patch\n"+x+"\n    try:\n        with patch('builtins.input') as input_mock:\n            input_mock.side_effect = FFF\n            print('**START**')\n"+text+"\n        print('GOODJOBANDSUCCESS')\n    except Exception as e:\n        print('####')\nsys.stdout.close()"
        #stdin별도처리
    if "stdin" in text:
        with open(f"D:/code_preprocessing/execute/{name}.py", "wt", encoding='utf-8') as f:
            f.write(text)
    elif ("stdin" in text) and ("input()" not in text):
        with open(f"D:/code_preprocessing/execute/{name}.py", "wt", encoding='utf-8') as f:
            text = f"import sys\nsys.stdin=open('D:/open/temp/temp.txt')\nfrom sys import stdin\nstdin=open('D:/open/temp/temp.txt')\nsys.stdout = open('D:/code_preprocessing/results/{name}.txt', 'w')\n" + text +"\nsys.stdout.close()"
            f.write(text)
        try:
            p = subprocess.run(['python', f"D:/code_preprocessing/execute/{name}.py"], timeout=1.5)
        except:
            continue       
    else:
        with open(f"D:/code_preprocessing/execute/{name}.py", "wt", encoding='utf-8') as f:
            f.write(text)
        try:
            p = subprocess.run(['python', f"D:/code_preprocessing/execute/{name}.py"], timeout=1.5)
        except:
            continue

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 17920/17920 [39:28<00:00,  7.57it/s]


In [17]:
#생성된 txt지우기
files = glob(f"D:/code_preprocessing/non_executable/*.py")
for f in files:
    os.remove(f)

In [18]:
problems= sorted(glob('D:/code_preprocessing/original/*/*.py'))
for problem in tqdm(problems):
    name = os.path.basename(problem)
    name = name.split(".")[0]
    folder = name.split("_")[0]
    if not os.path.exists(f"D:/code_preprocessing/results/{name}.txt"):
        shutil.copyfile(f"D:/code_preprocessing/execute/{name}.py", f"D:/code_preprocessing/non_executable/{name}.py")
    else:
        with open(f"D:/code_preprocessing/results/{name}.txt", "r") as f:
            text = f.read()
        if 'GOODJOBANDSUCCESS' in text:
            shutil.copyfile(f"D:/code_preprocessing/execute/{name}.py", f"D:/code_preprocessing/executable/{name}.py")
            with open(f"D:/code_preprocessing/results/{name}.txt", "r") as f:
                text = f.read()
            lines= text.split("\n")
            text_data = []
            for line in lines:
                if line == "**START**":
                    pass
                elif line == "####":
                    text_data = []
                elif line == "GOODJOBANDSUCCESS":
                    text_data = "\n".join(text_data)
                    with open(f"D:/code_preprocessing/executable/{name}.txt", "wt") as f:
                        f.write(text_data)
                    break
                else:
                    if line in text_data:
                        pass
                    else:
                        text_data.append(line)

        else:
            shutil.copyfile(f"D:/code_preprocessing/execute/{name}.py", f"D:/code_preprocessing/non_executable/{name}.py")

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 45101/45101 [04:35<00:00, 163.80it/s]


In [19]:
def Pav(lines):
    vline = []
    loop_on = 0
    for_num = 0
    variable_list = ''
    for line in lines:
        start = line.lstrip()  
        if "=" in line:
            index = line.find("=")
            vb = line[:index]
            f = vb.strip()
            if "," in f:
                FC = f.split(",")
                FCB = len(FC)
                tmp_num = 0
                for fc in FC:
                    fc = fc.strip()
                    c = re.findall(r'\w', fc)
                    if len(fc) == len(c):
                        tmp_num += 1
                if FCB == tmp_num:
                    variable = vb.lstrip()
                    blank = len(vb) - len(variable)
                    blank = vb[:blank]
                    vline.append(line)
                    rep = f"{blank}print('{variable}:',{variable})"
                    vline.append(rep)
                else:
                    vline.append(line)
            else:        
                c = re.findall(r'\w', f)
                d = re.findall(r'\w+\[\w+\]', f)
                if len(f) == len(c):
                    variable = vb.lstrip()
                    blank = len(vb) - len(variable)
                    blank = vb[:blank]
                    vline.append(line)
                    rep = f"{blank}print('{variable}:',{variable})"
                    variable_list += f"{variable}, "
                    vline.append(rep)
                elif len(f) == len(d):
                    variable = vb.lstrip()
                    blank = len(vb) - len(variable)
                    blank = vb[:blank]
                    vline.append(line)
                    rep = f"{blank}print('{variable}:',{variable})"
                    variable_list += f"{variable}, "
                    vline.append(rep)
                else:
                    vline.append(line)
        else:
            vline.append(line)
    vline = "\n".join(vline)
    return vline

In [20]:
#python3코드
problems= sorted(glob('D:/open/code/*/*.py'))
for problem in tqdm(problems):
    name = os.path.basename(problem)
    name = name.split(".")[0]
    folder = name.split("_")[0]
    text = preprocess_script(problem)
    lines = text.split("\n")
    text = Pav(lines)
    if not os.path.exists(f"D:/code_preprocessing/original/{folder}"):
        os.makedirs(f"D:/code_preprocessing/original/{folder}")
    with open(f'D:/code_preprocessing/original/{folder}/{name}.py', "wt", encoding='utf-8') as file:
        try:
            file.write(text)
        except:
            print(text)
            break

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 45101/45101 [00:30<00:00, 1475.75it/s]


In [21]:
problems= sorted(glob('D:/code_preprocessing/execute/*.py'))
for problem in tqdm(problems):
    name = os.path.basename(problem)
    name = name.split(".")[0]
    folder = name.split("_")[0]
    if not os.path.exists(f"D:/code_preprocessing/executable/{name}.txt"):
        shutil.copyfile(f"D:/code_preprocessing/original/{folder}/{name}.py", f"D:/code_preprocessing/non_executable/{name}.py")
        

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 45101/45101 [01:46<00:00, 424.68it/s]


In [22]:
#생성된 txt지우기
files = glob(f"D:/code_preprocessing/results/*.txt")
for f in files:
    os.remove(f)

In [23]:
problems= sorted(glob('D:/code_preprocessing/non_executable/*.py'))
for problem in tqdm(problems):
    name = os.path.basename(problem)
    name = name.split(".")[0]
    folder = name.split("_")[0]
    re_lines=[]
    with open(problem,'r',encoding='utf-8') as file:
        lines = file.readlines()
        for line in lines:
            re_lines.append("            "+line)
    text = '\n'.join(re_lines)
    text =  f"import sys\nsys.stdout = open(f'D:/code_preprocessing/results/{name}.txt', 'w')\nfrom unittest.mock import patch\n"+x+"\n    try:\n        with patch('builtins.input') as input_mock:\n            input_mock.side_effect = FFF\n            print('**START**')\n"+text+"\n        print('GOODJOBANDSUCCESS')\n    except Exception as e:\n        print('####')\nsys.stdout.close()"
        #stdin별도처리
    if "stdin" in text:
        with open(f"D:/code_preprocessing/execute/{name}.py", "wt", encoding='utf-8') as f:
            f.write(text)
    elif ("stdin" in text) and ("input()" not in text):
        with open(f"D:/code_preprocessing/execute/{name}.py", "wt", encoding='utf-8') as f:
            text = f"import sys\nsys.stdin=open('D:/open/temp/temp.txt')\nfrom sys import stdin\nstdin=open('D:/open/temp/temp.txt')\nsys.stdout = open('D:/code_preprocessing/results/{name}.txt', 'w')\n" + text +"\nsys.stdout.close()"
            f.write(text)
        try:
            p = subprocess.run(['python', f"D:/code_preprocessing/execute/{name}.py"], timeout=1.5)
        except:
            continue       
    else:
        with open(f"D:/code_preprocessing/execute/{name}.py", "wt", encoding='utf-8') as f:
            f.write(text)
        try:
            p = subprocess.run(['python', f"D:/code_preprocessing/execute/{name}.py"], timeout=1.5)
        except:
            continue

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 45100/45100 [2:08:45<00:00,  5.84it/s]


In [24]:
problems= sorted(glob('D:/code_preprocessing/original/*/*.py'))
for problem in tqdm(problems):
    name = os.path.basename(problem)
    name = name.split(".")[0]
    folder = name.split("_")[0]
    if not os.path.exists(f"D:/code_preprocessing/results/{name}.txt"):
        shutil.copyfile(f"D:/code_preprocessing/execute/{name}.py", f"D:/code_preprocessing/non_executable/{name}.py")
    else:
        with open(f"D:/code_preprocessing/results/{name}.txt", "r") as f:
            text = f.read()
        if 'GOODJOBANDSUCCESS' in text:
            shutil.copyfile(f"D:/code_preprocessing/execute/{name}.py", f"D:/code_preprocessing/executable/{name}.py")
            with open(f"D:/code_preprocessing/results/{name}.txt", "r") as f:
                text = f.read()
            lines= text.split("\n")
            text_data = []
            for line in lines:
                if line == "**START**":
                    pass
                elif line == "####":
                    text_data = []
                elif line == "GOODJOBANDSUCCESS":
                    text_data = "\n".join(text_data)
                    with open(f"D:/code_preprocessing/executable/{name}.txt", "wt") as f:
                        f.write(text_data)
                    break
                else:
                    if line in text_data:
                        pass
                    else:
                        text_data.append(line)

        else:
            shutil.copyfile(f"D:/code_preprocessing/execute/{name}.py", f"D:/code_preprocessing/non_executable/{name}.py")

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 45101/45101 [1:23:47<00:00,  8.97it/s]


In [ ]:
#Second Augmentation 

In [2]:
b1 = [['1','2','3','1','2','3','1','2','3','1'],['1 2', '1','2','3'],['1', '1 2','1 2','1 2'],['1 2','1 2','1','2'],['1','2','1 2','1 2'],['1 2 3'],['1 2 3','1'],['1','1 2 3'],['1 2 3 1'],['1 2 3 1','1'],['1','1 2 3 1'],['1 2 3 1 2'],['1 2 3 1 2 3']]
def inputs(input_data):
    x = f"for FFF in {input_data}:"
    return x
x = inputs(b1)

In [26]:
def Pav(lines):
    vline = []
    loop_on = 0
    for_num = 0
    variable_list = ''
    for line in lines:
        start = line.lstrip()
        Blank = len(line) - len(start)
        Blank = line[:Blank]
        if (start.startswith("for")) or (start.startswith("while")):
            if not loop_on:
                loop_on = 1
                for_num += 1
                line = line + f"#{for_num}"
            else:
                if variable_list:
                    vline.append(f"{Blank}try:\n{Blank}    print('{for_num} {variable_list}:',{variable_list[:-3]})\n{Blank}except:\n{Blank}    pass")
                    for_num += 1
                    line = line + f"#{for_num}"
                else:
                    pass
        else:
            if loop_on:
                if variable_list:
                    vline.append(f"{Blank}try:\n{Blank}    print('{for_num} {variable_list}:',{variable_list[:-3]})\n{Blank}except:\n{Blank}    pass")
                    loop_on = 0
                else:
                    pass
            else:
                pass
        
        
        if "=" in line:
            index = line.find("=")
            vb = line[:index]
            f = vb.strip()
            if "," in f:
                FC = f.split(",")
                FCB = len(FC)
                tmp_num = 0
                for fc in FC:
                    fc = fc.strip()
                    c = re.findall(r'\w', fc)
                    if len(fc) == len(c):
                        tmp_num += 1
                if FCB == tmp_num:
                    variable = vb.lstrip()
                    blank = len(vb) - len(variable)
                    blank = vb[:blank]
                    vline.append(line)
                    rep = f"{blank}print('{variable}:',{variable})"
                    vline.append(rep)
                else:
                    vline.append(line)
            else:        
                c = re.findall(r'\w', f)
                d = re.findall(r'\w+\[\w+\]', f)
                if len(f) == len(c):
                    variable = vb.lstrip()
                    blank = len(vb) - len(variable)
                    blank = vb[:blank]
                    vline.append(line)
                    rep = f"{blank}print('{variable}:',{variable})"
                    variable_list += f"{variable}, "
                    vline.append(rep)
                elif len(f) == len(d):
                    variable = vb.lstrip()
                    blank = len(vb) - len(variable)
                    blank = vb[:blank]
                    vline.append(line)
                    rep = f"{blank}print('{variable}:',{variable})"
                    variable_list += f"{variable}, "
                    vline.append(rep)
                else:
                    vline.append(line)
        else:
            vline.append(line)
    vline = "\n".join(vline)
    return vline

In [27]:
#생성 후 실행하는 코드
problems= sorted(glob('D:/code_preprocessing/original/*/*.py'))
for problem in tqdm(problems):
    name = os.path.basename(problem)
    name = name.split(".")[0]
    folder = name.split("_")[0]
    re_lines=[]
    with open(problem,'r',encoding='utf-8') as file:
        lines = file.readlines()
        for line in lines:
            re_lines.append("            "+line)
    text = '\n'.join(re_lines)
    text =  f"import sys\nsys.stdout = open(f'D:/code_preprocessing/results2/{name}.txt', 'w')\nfrom unittest.mock import patch\n"+x+"\n    try:\n        with patch('builtins.input') as input_mock:\n            input_mock.side_effect = FFF\n            print('**START**')\n"+text+"\n        print('GOODJOBANDSUCCESS')\n    except Exception as e:\n        print('####')\nsys.stdout.close()"
        #stdin별도처리
    if "stdin" in text:
        with open(f"D:/code_preprocessing/execute2/{name}.py", "wt", encoding='utf-8') as f:
            f.write(text)
    elif ("stdin" in text) and ("input()" not in text):
        with open(f"D:/code_preprocessing/execute2/{name}.py", "wt", encoding='utf-8') as f:
            text = f"import sys\nsys.stdin=open('D:/open/temp/temp.txt')\nfrom sys import stdin\nstdin=open('D:/open/temp/temp.txt')\nsys.stdout = open('D:/code_preprocessing/results/{name}.txt', 'w')\n" + text +"\nsys.stdout.close()"
            f.write(text)
        try:
            p = subprocess.run(['python', f"D:/code_preprocessing/execute2/{name}.py"], timeout=1.5)
        except:
            continue       
    else:
        with open(f"D:/code_preprocessing/execute2/{name}.py", "wt", encoding='utf-8') as f:
            f.write(text)
        try:
            p = subprocess.run(['python', f"D:/code_preprocessing/execute2/{name}.py"], timeout=1.5)
        except:
            continue

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 45101/45101 [1:36:16<00:00,  7.81it/s]


In [28]:
problems= sorted(glob('D:/code_preprocessing/original/*/*.py'))
for problem in tqdm(problems):
    name = os.path.basename(problem)
    name = name.split(".")[0]
    folder = name.split("_")[0]
    if not os.path.exists(f"D:/code_preprocessing/results2/{name}.txt"):
        shutil.copyfile(f"D:/code_preprocessing/execute2/{name}.py", f"D:/code_preprocessing/non_executable2/{name}.py")
    else:
        with open(f"D:/code_preprocessing/results2/{name}.txt", "r") as f:
            text = f.read()
        if 'GOODJOBANDSUCCESS' in text:
            shutil.copyfile(f"D:/code_preprocessing/execute2/{name}.py", f"D:/code_preprocessing/executable2/{name}.py")
            with open(f"D:/code_preprocessing/results2/{name}.txt", "r") as f:
                text = f.read()
            lines= text.split("\n")
            text_data = []
            for line in lines:
                if line == "**START**":
                    pass
                elif line == "####":
                    text_data = []
                elif line == "GOODJOBANDSUCCESS":
                    text_data = "\n".join(text_data)
                    with open(f"D:/code_preprocessing/executable2/{name}.txt", "wt") as f:
                        f.write(text_data)
                    break
                else:
                    if line in text_data:
                        pass
                    else:
                        text_data.append(line)

        else:
            shutil.copyfile(f"D:/code_preprocessing/execute2/{name}.py", f"D:/code_preprocessing/non_executable2/{name}.py")

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 45101/45101 [1:08:03<00:00, 11.04it/s]


In [3]:
b1 = [['0','1','2','0','1','2','0','1','2','0'],['0 1', '0','1','0'],['0', '0 1','0 1','0 1'],['0 1','0 1','0','1'],['0','1','0 1','0 1'],['0 1 2'],['0 1 2','0'],['0','0 1 2'],['0 1 2 3'],['0 1 2 3','0'],['0','0 1 2 3'],['0 1 2 3 0'],['0 1 2 3 0 1']]
def inputs(input_data):
    x = f"for FFF in {input_data}:"
    return x
x = inputs(b1)

In [4]:
#생성 후 실행하는 코드
problems= sorted(glob('D:/code_preprocessing/original/*/*.py'))
for problem in tqdm(problems):
    name = os.path.basename(problem)
    name = name.split(".")[0]
    folder = name.split("_")[0]
    re_lines=[]
    with open(problem,'r',encoding='utf-8') as file:
        lines = file.readlines()
        for line in lines:
            re_lines.append("            "+line)
    text = '\n'.join(re_lines)
    text =  f"import sys\nsys.stdout = open(f'D:/code_preprocessing/results3/{name}.txt', 'w')\nfrom unittest.mock import patch\n"+x+"\n    try:\n        with patch('builtins.input') as input_mock:\n            input_mock.side_effect = FFF\n            print('**START**')\n"+text+"\n        print('GOODJOBANDSUCCESS')\n    except Exception as e:\n        print('####')\nsys.stdout.close()"
        #stdin별도처리
    if "stdin" in text:
        with open(f"D:/code_preprocessing/execute3/{name}.py", "wt", encoding='utf-8') as f:
            f.write(text)
    elif ("stdin" in text) and ("input()" not in text):
        with open(f"D:/code_preprocessing/execute3/{name}.py", "wt", encoding='utf-8') as f:
            text = f"import sys\nsys.stdin=open('D:/open/temp/temp.txt')\nfrom sys import stdin\nstdin=open('D:/open/temp/temp.txt')\nsys.stdout = open('D:/code_preprocessing/results/{name}.txt', 'w')\n" + text +"\nsys.stdout.close()"
            f.write(text)
        try:
            p = subprocess.run(['python', f"D:/code_preprocessing/execute3/{name}.py"], timeout=1.5)
        except:
            continue       
    else:
        with open(f"D:/code_preprocessing/execute3/{name}.py", "wt", encoding='utf-8') as f:
            f.write(text)
        try:
            p = subprocess.run(['python', f"D:/code_preprocessing/execute3/{name}.py"], timeout=1.5)
        except:
            continue

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 45101/45101 [2:04:01<00:00,  6.06it/s]


In [5]:
problems= sorted(glob('D:/code_preprocessing/original/*/*.py'))
for problem in tqdm(problems):
    name = os.path.basename(problem)
    name = name.split(".")[0]
    folder = name.split("_")[0]
    if not os.path.exists(f"D:/code_preprocessing/results3/{name}.txt"):
        shutil.copyfile(f"D:/code_preprocessing/execute3/{name}.py", f"D:/code_preprocessing/non_executable3/{name}.py")
    else:
        with open(f"D:/code_preprocessing/results3/{name}.txt", "r") as f:
            text = f.read()
        if 'GOODJOBANDSUCCESS' in text:
            shutil.copyfile(f"D:/code_preprocessing/execute3/{name}.py", f"D:/code_preprocessing/executable3/{name}.py")
            with open(f"D:/code_preprocessing/results3/{name}.txt", "r") as f:
                text = f.read()
            lines= text.split("\n")
            text_data = []
            for line in lines:
                if line == "**START**":
                    pass
                elif line == "####":
                    text_data = []
                elif line == "GOODJOBANDSUCCESS":
                    text_data = "\n".join(text_data)
                    with open(f"D:/code_preprocessing/executable3/{name}.txt", "wt") as f:
                        f.write(text_data)
                    break
                else:
                    if line in text_data:
                        pass
                    else:
                        text_data.append(line)

        else:
            shutil.copyfile(f"D:/code_preprocessing/execute3/{name}.py", f"D:/code_preprocessing/non_executable3/{name}.py")

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 45101/45101 [1:07:24<00:00, 11.15it/s]
